# Plot critical values of alpha for different n

Limiting value $\alpha_{rdm}$: alpha for which the algorithm does not perform better than chance (mean error plus one standard deviation are $\ge 0.45$ ) <br/>
Critical value $\alpha_c$: alpha for which the performance is suddenly almost perfect (mean error plus one standard deviation $\le 0.10$). 

In [ ]:

from utils import *
from annealing_schedules import *
from mcmc import *
from experiments import *

import pickle as cPickle

### Run 10 experiments with Metropolis Hasting (constant $\beta=4.0$, $N=60k$) for each combination of $n \in [10,150]$ and $\alpha \in [0.05,1.5]$

In [ ]:
import time

start = time.time()

# Final and minimum error for a few different alphas.
n_vector = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150]
alphas = np.linspace(0.05, 1.5, 24)
N = 60000
n_exp = 10

beta_0 = 4.0

schedule = ConstantSchedule(beta_0)
b_v = schedule.get_schedule(N)


exps = defaultdict(defaultdict)
alpha_rdms_vec = defaultdict(defaultdict)
alpha_cs_vec = defaultdict(defaultdict)

alpha_rdms = defaultdict(defaultdict)
alpha_cs = defaultdict(defaultdict)

for n in n_vector:
    exps[n] = MultiExperiments(n, alphas, N, b_v, n_exp, MCMC.metropolis, )
    alpha_rdms_vec[n] = alphas[ (exps[n].get_stats("min_energy_errors")[0] + exps[n].get_stats("min_energy_errors")[1]) > 0.45 ]
    alpha_cs_vec[n] = alphas[ (exps[n].get_stats("min_energy_errors")[0] + exps[n].get_stats("min_energy_errors")[1]) < 0.10 ]
    alpha_cs[n] = alpha_cs_vec[n][0]
    alpha_rdms[n] = alpha_rdms_vec[n][-1]

end = time.time()
print(end - start)


### Save the results

In [ ]:
exp_error_data = defaultdict(defaultdict)
exp_parameter_data = defaultdict(defaultdict)

for n in n_vector:
    exp_error_data[n] = exps[n].get_stats("min_energy_errors")
    exp_parameter_data[n] = exps[n].parameters
    
fpickle = 'critical_alpha_metropolis_beta_4_10_150'

save_object = defaultdict(defaultdict)
save_object['n_vector'] = n_vector
save_object['alphas'] = alphas
save_object['N'] = N
save_object['n_exp'] = n_exp
save_object['b_v'] = b_v
save_object['exp_error_data'] = exp_error_data
save_object['exp_parameter_data'] = exp_parameter_data
save_object['alpha_rdms_vec'] = alpha_rdms_vec
save_object['alpha_cs_vec'] = alpha_cs_vec
save_object['alpha_rdms'] = alpha_rdms
save_object['alpha_cs'] = alpha_cs
    
file_handler = open("%s.pkl" %(fpickle),"wb")
cPickle.dump(save_object,file_handler)
file_handler.close()

    

## Critical alpha plot for $n \in [10,110]$ and $n \in [10,150]$ 

In [ ]:
def plot_critical_alphas(e_vectors, a_rnd_vector, a_c_vector, n_vector,  
                   x_axis_vector=None, title="", legend=[], xlabel="Time", fsave='' ):
    """Plots error over time (or the values in x_axis_vector). Inputs are
    lists of vectors to allow comparisons."""
    if x_axis_vector is None:
        x_axis_vector = np.arange(len(e_vectors))
    if not legend:
        legend = [" "] * len(e_vectors)
    fig = plt.figure(figsize=(20,10))
    ax1 = fig.gca()
    plt.hlines([0.1, 0.2, 0.3, 0.4, 0.5], x_axis_vector[0], x_axis_vector[-1],
               linestyles="--", color="lightgray")
    for i, e_vector in enumerate(e_vectors):
        e_vector_std = None
        if isinstance(e_vector, tuple):
            e_vector_std = e_vector[1]
            e_vector = e_vector[0]
        plt.errorbar(x_axis_vector, e_vector, yerr=e_vector_std, label=legend[i],
                    capthick=2, capsize=3)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel("Error")
    plt.ylim(0, 1)
    

    ax2 = ax1.twinx()
    ax2.plot(a_rnd_vector, n_vector, c='red', ls='--', label='α (random)')
    ax2.plot(a_c_vector, n_vector, c='red', ls=':', label='α (critical)')
    ax2.set_ylabel('n')

    #plt.legend()
    fig.legend(loc=1, bbox_to_anchor=(1,1), bbox_transform=ax1.transAxes)
    if (len(fsave) > 0):
        plt.savefig('%s.pdf' %(fsave))
        
fpickle = 'critical_alpha_metropolis_beta_4_10_150'

file = open("%s.pkl" %(fpickle),"rb")
save_object = cPickle.load(file)

n_vector = save_object['n_vector'] 
alphas = save_object['alphas'] 
N = save_object['N'] 
n_exp = save_object['n_exp'] 
b_v = save_object['b_v'] 
exp_error_data = save_object['exp_error_data'] 
exp_parameter_data = save_object['exp_parameter_data'] 
alpha_rdms_vec = save_object['alpha_rdms_vec'] 
alpha_cs_vec = save_object['alpha_cs_vec'] 
alpha_rdms = save_object['alpha_rdms'] 
alpha_cs = save_object['alpha_cs'] 
file.close()

a_rnd_v = []
a_c_v = []
for n in n_vector:
    a_c_v.append( alpha_cs[n] )
    a_rnd_v.append( alpha_rdms[n] )

fsave= 'Plots/critical_alpha_metropolis_beta_4_10_150'
fig = plot_critical_alphas([exp_error_data[100]], 
                           a_rnd_v, a_c_v, n_vector, x_axis_vector=alphas,
                           xlabel="alpha", legend=["Min. energy error"], fsave=fsave)



a_rnd_v = np.asarray(a_rnd_v)
a_c_v = np.asarray(a_c_v)
n_vector = np.asarray(n_vector)
a_rnd_v = a_rnd_v[ n_vector < 120 ]
a_c_v = a_c_v[ n_vector < 120 ]

fsave= 'Plots/critical_alpha_metropolis_beta_4_10_110'
fig = plot_critical_alphas([exp_error_data[100]], 
                           a_rnd_v, a_c_v, n_vector[n_vector < 120], x_axis_vector=alphas,
                           xlabel="alpha", legend=["Min. energy error"], fsave=fsave)

